One of the most usefull perception skills is being able to recognise objects. This allows creating robots that grasp objects and understand a bit better the world  around it.
* Recognise flat surfaces: This skill allows to detect places where objects.
* Recognise objects: once you know where to look you have to be able to recongnise different object in the scene and localise where the are from your robot.

"/base_controller/command" through Twist commands. You also can use other systems for moving the torso up and down and the head to look around. But in this object recognition only the movement of the base is really necessary.

### Table Top Detector

The first step in recognising objects is knowing where these objects are. For this you are going to use a port of the **tabletop_object_detector** package to be able to detect flats surfaces and represent that detection in RVIZ. (Object Recognition Kitchen)

So as usual the first step is to create your own object_recognition package:

**Execute in WebShell**
<br>
roscd: cd ../src
<br>
catkin_create_pkg my_object_recognition_pkg rospy object_recognition_core

You will have to add dependencies on each new package to need inside. 
<br>
So you are going to use this launch file:

<p style="color: white; background-color: green"> init_table_top.launch </p>

In [1]:
<launch>

    <arg name="tabletop_ork_file" value="$(find my_object_recognition_pkg)/conf/detection.table_fetch.ros.ork"/>
    
    <node pkg="object_recognition_core"
    type="detection"
    name="tabletop_server_node"
    args="-c $(arg tabletop_ork_file)"
    output="screen">
    </node>
    
</launch>

SyntaxError: invalid syntax (<ipython-input-1-3deeb299568a>, line 1)

<p style="color: white; background-color: green"> END init_table_top.launch  </p>

So as you can see, you are launching a binary called **detection** with a configuration file as argument. This file called **detection.tabletop_fetch.ros.ork** ist where all the input sensors and values for the table detection are set. Its a YAML file with a different extension .ork. So the first thing is create a *conf* directory inside your package. Then create this file:

<p style="color: white; background-color: green"> detection.tabletop_fetch.ros.ork  </p>

In [ ]:
source1:
    type: RosKinect
    module: 'object_recognition_ros.io'
    parameters:
        rgb_frame_id: '/head_camera_rgb_optical_frame'
        rgb_image_topic: '/head_camera/rgb/image_raw'
        rgb_camera_info: '/head_camera/rgb/camera_info'
        depth_image_topic: '/head_camera/depth_registered/image_raw'
        depth_camera_info: '/head_camera/depth_registered/camera_info'
        #
        #crop_enabled: True
        #x_min: -0.4
        #x_max: 0.4
        #y_min: -1.0
        #y_max: 0.2
        #z_min: 0.3
        #z_max: 1.5
        
sink1:
    type: TablePublisher
    module: 'object_recognition_tabletop'
        inputs: [source1]
            
pipeline1:
    type: TabletopTableDetector
    module: 'object_recognition_tabletop'
    inputs: [source1]
    outputs: [sink1]
    parameters:
        table_detector:
            min_table_size: 4000
            plane_threshold: 0.01
        #cluster:
        # table_z_filter_max: 0.35
        # table_z_filter_min: 0.025

<p style="color: white; background-color: green"> END  detection.tabletop_fetch.ros.ork   </p>

Of all the parameters you have here, the only relevant most of the times are the:
* rgb_frame_id: '/head_camera_rgb_optical_frame'
* rgb_image_topic: '/head_camera/rgb/image_raw'
* rgb_camera_info: '/head_camera/rgb/camera_info'
* depth_image_topic: '/head_camera/depth_registered/image_raw'
* depth_camera_info: '/head_camera/depth_registered/camera_info'

This set the correct image topics as inputs so that the recognition can be made. Once you have it, just execute the launch file.

**Execute in WebShell**
<br>
roslaunch my_object_recognition_pkg init_table_top.launch

The output you should get:

<img src="Images/R1.png" width="500">

And now open the RVIZ and add all the element you want to see (like the Camera element or PointCloud2 elements). To visualise the Table detection you will have to add **OrkTable** element. You select the topic where the table data is published, in this case **/table_array**. You can then check some options like "bounding_box" to have a bounding box around the detection or the "top" option to see what is being considered as the top of the surface.

Lets have a look at all the data textracted from **/table_array** topic:

<img src="Images/R2.png" width="500">

As you might see you can pinpoint the location of any surface detected or even filter the floor, because you know at waht hight each surface is.

**Exercise U3-1**

Create a script that makes the Fetch robot aproach to the table based on the data from the /table_array. This will allow you to look for a table and reach it for after searching for objects.

<p style="color: white; background-color: green"> Help Exercise U3-1   </p>

In [ ]:
#:/usr/bin/env python

import numpy 
import copy
import actionlib
import rospy
import time
from math import sin, cos
from moveit_python import (MoveGroupInterface, 
                          PlanningSceneInterface,
                          PickPlaceInterface)
from moveit_python.geometry import rotate_pose_msg_by_euler_angles

from control_msgs.msg import FollowJointTrajectoryAction, FollowJointTrajectoryGoal
from control_msgs.msg import PointHeadAction, PointHeadGoal
from geometry_msg.msg import Twist, Pose
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal
from moveit_msgs.msg import JointTrajectory, JointTrajectoryPoint
from nav_msgs.msg import Odometry

#Move base using navigation stack
class MoveBasePublisher(object):
    
    def init (self):
        self.move_base_publisher = rospy. Publisher('/base_controller/command', Twist, queue_size=1)
        self.odom_subs = rospy.Subscriber('/odom', Odometry, self.odom_callback)
        self.actual_pose = Pose()
        
    def odom_callback(self, msg):
        self.actual_pose = msg.pose.pose
        
    def move_to(self, twist_object):
        self._move_base_publisher.publish(twist_object)
        
    def stop(self):
        twist_object = Twist()
        self.move_to(twist_object)
        
    def move_forwards_backwards(self, speed_ms, position_x, allow_error=0.1):
        """
        distance_metres: positive is go forwards, negative ga backwards
        """
        rate = rospy.Rate(5)
        move_twist = Twist()
        
        start_x = self.actual_pose.position.x
        direction_speed = numpy.sign(position_x - start.x)
        move_twist.linear.x = direction_speed * speed_ms
        
        in_place = False
        while not in_place:
            self.move_to(move_twist)
            x_actual = self.actual_pose.position.x
            print (x_actual)
            
            if abs(position_x - x_actual) <= allowed_error:
                print ("Reached position")
                break
                
            rate.sleep()
        self.stop()
        
# Move base using naviagtion stack
class MoveBaseClient(object):
    
    def __init__(self):
        self.client = actionlib.SimpleActionClient("move_base", MoveBaseAction)
        rospy.loginfo("Waiting for move_base...")
        self.client.wait_for_server()
        
    def goto(self, x, y, theta, frame="map"):
        move_goal = MoveBaseGoal()
        move_goal.target_pose.pose.position.x = x
        move_goal.target_pose.pose.position.y = y
        move_goal.target_pose.pose.orientation.z = sin(theta/2.0)
        move_goal.target_pose.pose.orientation.w = cos(theta/2.0)
        move_goal.target_pose.header.frame_id = frame
        move_goal.target_pose.header.stamp = rospy.Time.now()
        
        #TODO wait for things to work
        self.client.send_goal(move_goal)
        self.client.wait_for_result()
        
#Send a trajectory to controller
class FollowTrajectoryClient(object):
    
    def __init__(self, name, joint_names):
        self.client = actionlib.SimpleActionClient("%s/follow_joint_trajectory" % name,
                                                  FollowJointTrajectoryAction)
        rospy.loginfo("Waiting for %s..." % name)
        self.client.wait_for_server()
        self.joint_names = joint_names
        
    def move_to(self, positions, duration=5.0):
        if len(self.joint_names) != len(positions):
            print("Invalid trajecotry position")
            return False
        trajectory = JointTrajectory()
        trajectory.joint_names = self.joint_names
        trajectory.points.append(JointTrajectoryPoint())
        trajectory.points[0].position = positions
        trajectory.points[0].velocities = [0.0 for _ in positions]
        trajectory.points[0].accelerations = [0.0 for _ in positions]
        trajectory.points[0].time_from_start = rospy.Duration(duration)
        follow_goal = FollowJointTrajectoryGoal()
        follow_goal.trajectory = trajectory
        
        self.client.send_goal(follow_goal)
        self.client.wait_for_result()
        
# Point the head using controller
class PointHeadClient(object):
    
    def __init__(self):
        self.client = actionlib.SimpleActionClient("head_controller/point_head", PointHeadAction)
        rospy.loginfo("Waiting for head_controller...")
        self.client.wait_for_server()
        
    def look_at(self, x, y, z, frame, duration=1.0):
        goal = PointHeadGoal()
        goal.target.header.stamp = rospy.Time.now()
        goal.target.header.frame_id = frame
        goal.target.point.x = x 
        goal.target.point.y = y
        goal.target.point.z = z
        goal.min_duration = rospy.Duration(duration)
        self.client.send_goal(goal)
        self.client.wait_for_result()
        
# Tools for grasping
class GraspingClient(object):
    
    def __init__(self):
        self.scene = PlanningSceneInterface("base_link")
        self.pickplace = PickPlaceInterface("arm", "gripper", verbose=True)
        self.move_group = MoveGroupInterface("arm", "base_link")
        
    def getGraspableCube(self):
        
        #nothing detected
        return None, None
    
    def getSupportSurface(self, name):
        for surface in self.support_surfaces:
            if surface.name == name:
                return surface
        return None
    
    def getPlaceLocation(self):
        pass
    
    def pick(self, block, grasps):
        success, pick_result = self.pickplace.pick_with_retry(block.name,
                                                             grasps,
                                                             support_name=block.support_surface,
                                                             scene=self.scene)
        self.pick_result = pick_result
        return success
    
    def place(self, block, pose_stamped):
        places= list()
        l = PlaceLocation()
        l.place_pose.pose = pose_stamped.pose
        l.place_pose.header.frame_id = pose_stamped.header.framed_id
        
        # copy the posture, approach and retreat from the grasp used
        l.pose_place_posture =  slef.pick_result.grasp.pre_grasp_posture
        l.pose_place_approach =  slef.pick_result.grasp.pre_grasp_approach
        l.pose_place_retreat =  slef.pick_result.grasp.post_grasp_retreat
        # create another several places, rotate each by 90 degrees in yaw direction
        l.place_pose.pose = rotate_pose_msg_by_euler_angles(l.place_pose.pose, 0, 0, 1.57)
        places.append(copy.deepcopy(l))
        l.place_pose.pose = rotate_pose_msg_by_euler_angles(l.place_pose.pose, 0, 0, 1.57)
        places.append(copy.deepcopy(l))
        l.place_pose.pose = rotate_pose_msg_by_euler_angles(l.place_pose.pose, 0, 0, 1.57)
        places.append(copy.deepcopy(l))
        
        succes, palce_result = self.pickplace.place_with_retry(block.name, 
                                                              places,
                                                              scene=self.scene)
        return success
    
    def stuk(self):
        joints = ["shoulder_pan_joint", "shoulder_lift_joint", "upperarm_roll_joint",
                 "elbow_flex_joint", "forearm_roll_joint", "wrist_flex_joint", "wrist_roll_joint"]
        pose = [1.32, 1.40, -0.2, 1.72, 0.0, 1.66, 0.0]
        while not rospy.is_shutdown():
            result = self.move_group.moveToJointPosition(joints, pose, 0.02)
            if result.error_code.val == MoveItErrorCodes.SUCCESS:
                return
        
def demo():
    # Create a node
    rospy.init_node("face_recognition_demo_node")
    
    # Make sure sim time is working
    while not rospy.Time.now():
        pass
    
    move_base_pub = MoveBasePublisher()
    rospy.loginfo("Start Sequence complete")
    rospy.loginfo("Move to position ...")
    move_base_pub.move_forwards_backwards(speed_ms=0.3, position_x=1.0, allowed_error=0.1)
    
if __name__ == "__main__":
    demo()
    
    

### 2d and 3d Object Finder

This is basic approach for detecting objects, although you can already differenciate objects and localise them. For this you will use "find-object-2d" package.

The first step is to get closer to the table to see the objects as close as possible to make better detections. You can use your recently created script to move the Fecth robot closer to the table.

##### 2d Object Finder

Here you have an exmaple of the launch you would have to launch to start the basic system:

<p style="color: white; background-color: green"> start_find_object_2d.launch   </p>

In [ ]:
<?xml version="1.0" encoding="UTF-8"?>

<launch>
    <arg name="camera_rgb_topic" default="/head_camera/rgb/image_raw" />
    <node name="find_object_2d_node" pkg="find_object_2d" type="find_object_2d" output="screen">
        <remap from="image" to="$(arg camera_rgb_topic)"/>
    </node>
</launch>

<p style="color: white; background-color: green"> END   </p>

You have only to set the RGB camera image souce and the system is ready to go. In this case its **/head_camera/rgb/Image_raw**. Once you launch it you should go to The Graphical Tools and see something similar to this:

<img src="Images/R3.png" width="500">

Fantastic! Now You can add as many objects as you want, or even turn the object around and take images from different points of view. This will make it detect the object all the time, only bare in mind that without the proper filtering, the system will consider it as different objects.

So the last step is to save all the objects added. There are mainly 2 ways:
* Saving the Objects as images: File --> Save_Objects. This will save in a folder all the images taken
* Saving the Whole session: File -->Save_Session. This will save a binary with all the images and settings. This is the most compact way of doing it, although you wont have acces to the images of the objects. Depends on your needs.


You can also save the object images directly by rightclicking on them but <font color=red> BEWARE </font> that if you do so, these images can't be used for later delections, because they aren't mirrored, therefore the recognitin wont work. You can see an example of it here, its the sam image but one is the manualy saved version that is not mirrored and isn't detected, while the other one is correctly saved.

So once you have your session or your images stored, you need to be able to always start an object recognition session with all that stored data. To do so you have the folloeing options:

For saved sessions in my_object_revognition_pkg, in the directory saved_pictures2d:

In [ ]:
<launch>
    <arg name="camera_rgb_topic" default="/head_camera/rgb/image_raw" />
    <!-- Nodes -->
    <node name="find_object_2d_node" pkg="find_object_2d" type="find_object_2d" output="screen">
        <remap from="image" to="$(arg camera_rgb_topic)"/>
        <param name="gui" value="true" type="bool"/>
        <param name="session_path" value="$(find my_object_recongnition_pkg)/saved_pictures2d/coke_session.bin" type="str"/>
        <param name="settings_path" value="~/.ros/find_object_2d.ini" type="str"/>
    </node>
</launch>

For saved images in my_object_recognition_pkg, in the directory saved_pictures2d:

In [ ]:
<launch>
    <arg name="camera_rgb_topic" default="/head_camera/rgb/image_raw" />
    <!-- Nodes -->
    <node name="find_object_2d_node" pkg="find_object_2d" type="find_object_2d" output="screen">
        <remap from="image" to="$(arg camera_rgb_topic)"/>
        <param name="gui" value="true" type="bool"/>
        <param name="objects_path" value="$(find my_object_recongnition_pkg)/saved_pictures2d/coke_session.bin" type="str"/>
        <param name="settings_path" value="~/.ros/find_object_2d.ini" type="str"/>
    </node>
</launch>

**Exercise U3-2**

Create a launch file that starts the 2d object recognition through a saved seddion where you have saved the coke can object.

##### Move and Spawn objects

You have to test your object_recognition system with the same object in different positions or even in movement.

You also need to test it with various objects in scene to be sure that it doesn't mistake a human head with a coke.

Here you are going to learn how to move around objects in a Gazebo scene, and spawn new ones also.

To make an object move in a scene there are two steps:

* **Make the object movable in Gazebo**: for that you wil use the move_model.launch from our spawn_robot_tools_pkgs. THis makes available a topic called "name_object/cmd_vel" on which you can then publish and move the model around.
* **Publish in the correct topic** to move the object through the keyboard.

If you want to know exactly how this works, see TF and URDF Tutorium.

<p style="color: white; background-color: green"> make_cokecan_moveable.launch</p>

In [ ]:
<?xml version="1.0" encoding="UTF-8"?>

<launch>
    <arg name="robot_name" default="coke_can" />
    
    <include file="$(find spawn_robot_tools_pkg)/launch/move_model.launch">
        <arg name="robot_name" value="$(arg robot_name)">
    </include>
</launch>

<p style="color: white; background-color: green"> End   </p>

<p style="color: white; background-color: green"> MOVE_COKE_KEYBOARD.LAUNCH   </p>

In [ ]:
<?xml version="1.0" encoding="UTF-8"?>

<launch>
    <arg name="robot_name" default="coke_can" />
    <node name="$(arg robot_name)_twist_keyboard" pkg="spwan_robot_tools_pkg" type="model_twist_keyboard.py" 
    args="$(arg robot_name)" output="screen"/>
</launch>    

<p style="color: white; background-color: green"> End  </p>

To move other objects in the scence just change the name of the model. 

To know the name of the model has in Gazebo you can ask the gazebo service:

In [ ]:
Execute in WebShell #1

rosservice call /gazebo/get_world_properties

So now how do you spawn new object in the scene?
<br>
There are many ways, ***we highly recommend you to learn TF and URDF robot creation***.

This method needs you to have the sdf files of the models on your package and the whole model installed in the .gazebo/models path. For simplicity we have granted access to three of them: beer, coke_can and hammer. You really only need the sdf files of each model, the rest will be retrieved from the gazebo path, not your package. You can copy them to your package by:

Once you have them just execute the following launch file to spawn the model where you wish, probably in the table location.
<br>
For Reference, the coke_can is spawned in the center of the table at XYZ=(-2.0, 0.0, 0.8).
<br>
Bare in mind that if there is already an existing model with the same name, you wont be able to spawn it.

<p style="color: white; background-color: green"> spawn_coke.launch   </p>

In [ ]:
<launch>

    <arg name="sdf_robot_file" value="$(find my_object_recognition_pkg)/models/coke_can/model.sdf" />
    <arg name="robot_name" value="coke_can" />
    <arg name="x" default="-2.0" />
    <arg name="y" default="0.2" />
    <arg name="z" default="0.8" />
    <arg name="yaw" default="0.0" />
    
    <include file="$(find spawn_robot_tools_pkg)/launch/spawn_sdf.launch">
        <arg name="sdf_robot_file" value="$(arg sdf_robot_file)" />
        <arg name="robot_name" default="$(arg robot_name)" />
        <arg name="x" default="$(arg x)" />
        <arg name="y" default="$(arg y)" />
        <arg name="z" default="$(arg z)" />
        <arg name="yaw" default="$(arg yaw)" />
    </include>
    
</launch>    

<p style="color: white; background-color: green"> End   </p>

So in conclusion to spawn an object, for example the coke_can, and make it mobable at the same time you have to execute this launch file:

<p style="color: white; background-color: green"> spawn_ready_coke.lauch   </p>

In [ ]:
<launch>
    <!-- Spawn CokeCan Model -->
    <include file="$(find my_object_recognition_pkg)/launch/spwan_coke.launch" />
    <!-- Make CokeCan Movable -->
    <include file="$(find my_object_recognition_pkg)/launch/make_cokecan_movable.launch" />
</launch>

<p style="color: white; background-color: green"> End   </p>

And afterwards in another terminal, execute the **move_coke_keyboard.launch**.

And to delete the object, just RIGHTCLICK ON it and Delete it.

<p style="color: white; background-color: orange"> Exercise U3-3  </p>

Do the following:
* Delete the existing cocke model of the scene.
* Spawn a new coke_can on the table.
* Move the coke can around and see how robust is the detection system with only one picture.
* Then make as many pictures as you need to never losse track of the object. The best method is to pivot in place the coke can until it stops detecting. Then add it as a new object, and repeat the process until it detects it 360 degrees.

<p style="color: white; background-color: orange"> End  </p>

You should at the end have a detection squeme similar to this one:

<img src="Images/R4.png" width="500">

<p style="color: white; background-color: orange"> Help Exercise U3-4   </p>

Now do the same as in Exercise U3-3 but wih a beer can and a hammer.
<br>
For this you will have to spawn and move them first.
<br>
Remmember to save the session at the end so that you dont losse all your object images.

<p style="color: white; background-color: orange"> End  </p>

**3d Object Detection**

So now the only thing missing is having the position of the objects in 3d space to be able to grasp them. To know how to grasp an object, please do the *Manipulation course*.

The only real difference with the 2d detection will be the sensors involved and the fact that the ObjectPoseStamped will be transformed into TFs.

<font color="red"> **Warming** </font>

You have to create another round of session fotos in the 3d system, because otherwise the detection wont work as well as they should. Specially for the TF transformations.

<p style="color: white; background-color: green"> start_find_object_3d_session.launch   </p>

In [ ]:
<launch>
    
    <node name="find_object_3d" pkg="find_object_3d" type="find_object_3d" output="screen">
        <param name="gui" value="true" type="bool" />
        <param name="settings_path" value="~/.ros/find_object_2d.ini" type="str" />
        <param name="subscribe_depth" value="true" type="bool" />
        <param name="session_path" value="$(find my_object_recognition_pkg)/saved_pictures2d/coke_session.bin"
        type="str" />
        <param name="objects_path" value="" type="str" />
        <param name="object_prefix" value="object" type="str" />
        
        <remap from="rgb/image_rect_color" to="/head_camera/rgb/image_raw" />
        <remap from="depth_registered/image_raw" to="/head_camera/depth_registered/camera_info" />
    </node>
    
    <!-- Example of tf synchronisation with the objectsStamped message -->
    <node name="tf_example" pkg="find_object_2d" type="tf_example" output="screen" />
        <param name="map_frame_id" value="/map" type="string" />
        <param name="object_prefix" value="object" type="str" />
    </node>
    
</launch>   

<p style="color: white; background-color: green"> End   </p>

Launching this you should then get the TF published of the object detected. If you have multiple images of the same object you will get multiple frame of objects, that's up to you to filter them.

You can also see the object detected by executed by executing in another terminal while the prior launch is working, the following command:

<p style="color: white; background-color: orange"> Exercise U3-4   </p>

Do the following:
* Launch the 3d object detector with a session with only one image per object. This way you wont get a clutter of TF frames.
* Move the objects around through a python script. This way you will be able to see how the TFs change and how accurate they are in relation to the real position of the objects.

<p style="color: white; background-color: orange"> End</p>

The TFs appearing can be lowered by lowering the time you sonsifer a TF obsolete. Because most of these TFS are from previous detections that you stay there for a while until they are old enough to consider them irrelevant.

<p style="color: white; background-color: orange"> Extra Exercise U3-5   </p>

Do the following: 
* Spawn two objects in the table and make them move on the table.
* Detect one of the objects.
* Track that object and make the Fetch Robot track it with the head to keep the object in the center of the view.

<p style="color: white; background-color: orange"> End   </p>

**Congratulations!**